# Q1. (40 points) Feature extraction and matching

   (use the images from https://sourceforge.net/projects/adobedatasets.adobe/files/adobe_panoramas.tgz/download)  
    a. (10 points) Select and implement one of the point feature detectors we have explained or use other methods you learned about. (Make sure to implement the feature detector yourself). Explain your selected detector and show the results.   
    b. (20 points) Implement a simple feature matching by using two feature descriptors of your choice (you can use the available feature descriptors in OpenCV or Matlab). Compare the two feature descriptors and the matching results on a few different images.  
    c. (10 points) Instead of finding feature points independently in multiple images and then matching them, find features in the first image of a video or image sequence and then re-locate the corresponding points in the next frames using search and gradient descent (Shi and Tomasi 1994). When the number of tracked points drops below a threshold or new regions in the image become visible, find additional points to track.

## a.  Harris Corner Point detection

The feature I implemented is Harris corner point. The defail info can be found at [wikipedia](https://en.wikipedia.org/wiki/Corner_detection#The_Harris_&_Stephens_/_Shi%E2%80%93Tomasi_corner_detection_algorithms)

The key function is `get_harris_points`, the detail code can be found in assignment_code.py ane extra function `display_corner_points` is written for display finding point on image as red point.

Python code
```python
def get_harris_points(I, k=0.05):

    # check image and convert to gray and normalize
    if len(I.shape) == 3 and I.shape[2] == 3:
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    if I.max() > 1.0:
        I = I / 255.0

    # Step 1 calcualte Axx, Axy and Ayy
    
    # Step 1.1 calculate Ix, Iy
    # apply soble filter for quick calculation
    filter_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]) / \
        8.0  # sobel filter for x derivative
    filter_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]]) / \
        8.0  # sobel filter for y derivative

    Ix = imfilter(I, filter_x)
    Iy = imfilter(I, filter_y)
    Ixx = Ix * Ix
    Iyy = Iy * Iy
    Ixy = Ix * Iy

    window = np.ones((3, 3))

    Axx = imfilter(Ixx, window)
    Axy = imfilter(Ixy, window)
    Ayy = imfilter(Iyy, window)

    # Step 2 calculate response
    # determinant
    detA = Axx * Ayy - Axy ** 2
    # trace
    traceA = Axx + Ayy
    # response
    response = detA - k * traceA ** 2

    # step 3. Get points location(x,y)
    points = corner_peaks(response,min_distance=4)
    # points = get_coordinate(response,I.shape)

    return points
```


In [9]:
# showing result
import assignment_code as ac
import numpy as np
from pprint import pprint
import cv2
from skimage.feature import corner_peaks, corner_subpix
import matplotlib.pyplot as plt

image_name = "data/carmel/carmel-00.png"
# image = cv2.imread(image_name,cv2.IMREAD_GRAYSCALE)
# plt.imshow(image, cmap='gray', vmin=0, vmax=255)
# plt.title("original")
# plt.show()


I = cv2.imread(image_name)
grey = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
grey = grey / 255.0
points = ac.get_harris_points(grey)
output_name = f"./output/a_harris_output.jpg"
ac.display_corner_points(I,points,output_name)
image2 = cv2.imread(output_name,cv2.IMREAD_GRAYSCALE)

# plt.imshow(image2, cmap='gray', vmin=0, vmax=255)
# plt.title("Image with Harris Points")
# plt.show()

# fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 7), dpi=80, sharex=True, sharey=True)
# ax[1].imshow(image, cmap='gray')
# ax[0].imshow(image2, cmap='gray')

Disply Result

The harris point is marked to red color

Original Image  
![org_image](./data/carmel/carmel-00.png)

Image with Harris Points  
![harris_image](./output/a_harris_output.jpg)

## B. Feature Matching
Implement a simple feature matching by using two feature descriptors of your choice (you can use the available feature descriptors in OpenCV or Matlab). Compare the two feature descriptors and the matching results on a few different images. 
